### **Repo Selection**

In [8]:
# TODO: Discuss idea with Professor
# Q. What high prompt-density repositories should we be looking at? And, for which parsers?

# USING DEFAULT PARSER for now
import json
with open('../parse_data/repo_to_prompts.json', 'r') as f:
    data = json.load(f)

all_prompts = []
high_density_repo_prompts = []
for repo in data:
    all_prompts.extend(data[repo])
    if len(data[repo]) > 10:
        high_density_repo_prompts.extend(data[repo])

print(len(all_prompts))
print(len(high_density_repo_prompts))

1127
577


### **Text Analysis**

In [88]:
import pandas as pd
df = pd.DataFrame(high_density_repo_prompts)
df.columns = ['prompts']

df.to_csv('high_density_repo_prompts.csv', index=False)

count = 0
for prompt in high_density_repo_prompts:
    if "assistant" in prompt.lower() or "assistent" in prompt.lower():
        count += 1
        # print(prompt)
count

51

### ❌ **Spell Check** (FAILED) (WIP)

In [84]:
import spacy
from spellchecker import SpellChecker
from spacy.language import Language
import re

misspelled_freq = {}
@Language.component("spellcheck_component")
def spellcheck_component(doc):
    # Initialize the SpellChecker
    spell_checker = SpellChecker()
    spell_checker.word_frequency.load_words(
        ["f", "langchain", "openai", "markdown", "json", "api", "endpoint", "covid", "docstore", "chatopenai", 
         "append", "schema", "gpt", "", " ", "yyyy", "mm", "dd", "sql", "st", "dataset", "Zelenskyy"] + 
        #  These are things picked up by spaCy due to the use of contractions
        ["s", "b", "d", "m", "t"] +
        []
    )

    misspelled = spell_checker.unknown([token.text for token in doc if token.is_alpha])

    # Go through the tokens and suggest corrections for misspelled words
    # for token in doc:
    #     if token.text in misspelled:
    #         suggestions = spell_checker.candidates(token.text)
    #         if suggestions:
    #             print(f"Original: {token.text}")
    #             print(f"Suggestions: {suggestions}")
    for word in misspelled:
        misspelled_freq[word] = misspelled_freq.get(word, 0) + 1

    # if "b" in misspelled:
    #     print(doc)
    #     print(misspelled)
    #     print("###########################################################")

    # if len(misspelled) > 0:
    #     print(f"Misspelled: {misspelled}")

    # Return the doc following spaCy's pipeline component requirements
    return doc

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Add spell-checking component to the pipeline
nlp.add_pipe("spellcheck_component")

text = "The quick brwn fox jmps over the lze dog."

# Process the text with spaCy and check spelling
nlp(text)

for prompt in high_density_repo_prompts:
    modified_string = re.sub(r'((\{+)(\s*)(.*?)(\s*)(\}+))', ' ', prompt)  # remove {variable}
    modified_string = re.sub(r'\\n', ' ', modified_string)  # to avoid "\nTonight"
    modified_string = re.sub(r"[^\w']+", " ", modified_string)  # to avoid "good-morning" but words like keep "don't"
    modified_string = re.sub(r"_+", " ", modified_string)  # to avoid "______" or "good__morning"
    doc = nlp(modified_string)

print(sorted(misspelled_freq.items(), key=lambda x: x[1], reverse=True))

[('金额数', 19), ('销售方式', 19), ('是否含申购费', 19), ('限额项目', 19), ('单位', 19), ('sk', 15), ('gps', 15), ('severability', 15), ('wilbert', 15), ('alzheimer', 15), ('unenforceability', 15), ('arpa', 15), ('invalidity', 15), ('h', 15), ('startswith', 14), ('syntactically', 14), ('argilla', 13), ('huggingface', 12), ('argillatrainer', 12), ('trainingtask', 12), ('feedbackdataset', 12), ('元', 12), ('含', 12), ('standalone', 12), ('v', 12), ('sqlresult', 12), ('sqlquery', 12), ('llm', 11), ('openapi', 11), ('delimited', 11), ('func', 10), ('demogpt', 10), ('g', 10), ('integrations', 10), ('ux', 10), ('str', 9), ('dict', 9), ('config', 9), ('py', 9), ('ros', 9), ('pl', 9), ('n', 9), ('params', 9), ('coreference', 8), ('db', 7), ('https', 7), ('streamlitcallbackhandler', 6), ('llmmathchain', 6), ('duckduckgosearchrun', 6), ('verbose', 6), ('llmchain', 6), ('申购最低额', 6), ('追加申购最低额', 6), ('份', 6), ('x', 6), ('endpoints', 6), ('attr', 6), ('comparators', 6), ('comparator', 6), ('msgs', 5), ('conversationbuf

### 🧩 **Searching Patterns** (WIP)

In [91]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
patterns = [
    [{"LOWER": "assistant"}],
]

matcher.add("GPPT-3", patterns)

def find_pattern(text, patterns):
    doc = nlp(text)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        print(match_id, string_id, start, end, span.text)

    return matches

count = 0
for p in high_density_repo_prompts:
    if find_pattern(p, patterns):
        count += 1
print(count)

14194311245358984270 GPPT-3 70 71 assistant
14194311245358984270 GPPT-3 84 85 assistant
14194311245358984270 GPPT-3 73 74 assistant
14194311245358984270 GPPT-3 87 88 assistant
14194311245358984270 GPPT-3 8 9 assistant
14194311245358984270 GPPT-3 17 18 assistant
14194311245358984270 GPPT-3 30 31 assistant
14194311245358984270 GPPT-3 113 114 assistant
14194311245358984270 GPPT-3 264 265 assistant
14194311245358984270 GPPT-3 13 14 assistant
14194311245358984270 GPPT-3 25 26 assistant
14194311245358984270 GPPT-3 359 360 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 27 28 Assistant
14194311245358984270 GPPT-3 5 6 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 6 7 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 60 61 Assistant
14